In [1]:
from datetime import datetime
res = !gcloud config get core/project
PROJECT_ID = res[0]
CONTAINER_IMAGE_NAME="impressions"
GCP_REGION='us-central1'
TAG_NAME = 'masterv9'

# KAGGLE_KEY=''
CONTAINER_IMAGE_NAME_DATA_PREP = f"{CONTAINER_IMAGE_NAME}-data-preparation"
CONTAINER_IMAGE_NAME_FINE_TUNE = f"{CONTAINER_IMAGE_NAME}-trainning"
CONTAINER_IMAGE_NAME_RUN_APP = f"{CONTAINER_IMAGE_NAME}-running-app"
CONTAINER_IMAGE_NAME_PIPELINE = f"{CONTAINER_IMAGE_NAME}-pipeline-app"
CONTAINER_IMAGE_RUNNING_APP = f"{CONTAINER_IMAGE_NAME}-running-app"

In [2]:
import os
print(os.getcwd())

/home/andre/impressions


In [3]:
from components.data import youtube_captions, prepare_text_data
from components.utils import gcs_utils
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
url =  'https://www.youtube.com/watch?v=OuF9weSkS68&list=PL0iVR8sl9TiWqj_JmVjEgAzl4YhtTt9Wf' #'https://www.youtube.com/watch?v=pCX_3p40Efc'
id_type = 'channel'
language = 'en'
blob_name = 'ColdFusion' # 'SentdexChannel'
bucket_name = 'metal-sky-419309-videos-v1'
development_percentage = 1
train_percentage = .9
tokenizer_type = 'gpt-4'
max_number_videos = 20

config = dict(bucket_name=bucket_name, url=url, id_type=id_type, language=language, blob_name=blob_name,
              max_number_videos=max_number_videos, development_percentage=development_percentage, train_percentage=train_percentage,
              tokenizer_type=tokenizer_type)
for a, b in config.items():
    print(a, b)

bucket_name metal-sky-419309-videos-v1
url https://www.youtube.com/watch?v=OuF9weSkS68&list=PL0iVR8sl9TiWqj_JmVjEgAzl4YhtTt9Wf
id_type playlist
language en
blob_name ColdFusion
max_number_videos 20
development_percentage 1
train_percentage 0.9
tokenizer_type gpt-4


In [5]:
videos = youtube_captions.get_videos_and_captions(config['url'], config['id_type'], config['language'])
if videos:
    gcs_utils.save_dict_to_gcs(videos, bucket_name=config['bucket_name'], blob_name = config['blob_name'])
else:
    logger.error("No video data to save.")

In [6]:
from components.config.Config import ConfigGPT
config = ConfigGPT()
config.dataset = blob_name
print(config)

Simon
Configuration:
out_dir: out
eval_interval: 200
log_interval: 1
eval_iters: 200
eval_only: False
always_save_checkpoint: True
init_from: scratch
dataset: ColdFusion
block_size: 84
batch_size: 32
n_layer: 4
n_head: 8
n_embd: 72
dropout: 0.0
bias: False
learning_rate: 0.0006
max_iters: 600
decay_lr: True
min_lr: 6e-05
grad_clip: 1.0
seed_offset: 0
device: cpu
dtype: float16
compile: True
model_name: Impressions



In [7]:

prepare_text_data.process_captions_and_prepare_data(bucket_name, blob_name, development_percentage, train_percentage, tokenizer_type, language)

Blob ColdFusion read successfully.
Video c49WRDVCvpo does not have captions in Spanish.
Video Cn4pNfudPHc does not have captions in Spanish.
Video 2qMl8JkJ5hg does not have captions in Spanish.
Video 3DUb1P5HjlQ does not have captions in Spanish.
Master caption length: 949675
The tokenizer type is : gpt-4
This is the known token idx : [1, 4, 6, 8, 11]
This is the unknown token idx : [27, 29, 32983]
This is the stoi type 12532
This is the itos type 12532
This is the vocab size 12532
this video is brought to you by ground news hi welcome to another episode of Cold Fusion we know for a fact that what all of these systems do every single one is it exploits our own natural tendencies in human beings to get and want feedback and that feedback chemically speaking is the release of dopamine in your brain and so what these feedback loops do and they exist everywhere in Call of Duty in other video games in social networking sites they get you to react and I think that if you get too desensitized

In [8]:
import torch
from components.training.train import Trainer

print(config)
print("Training...")
trainer = Trainer(config)
model = trainer.train()


Simon
Configuration:
out_dir: out
eval_interval: 200
log_interval: 1
eval_iters: 200
eval_only: False
always_save_checkpoint: True
init_from: scratch
dataset: ColdFusion
block_size: 84
batch_size: 32
n_layer: 4
n_head: 8
n_embd: 72
dropout: 0.0
bias: False
learning_rate: 0.0006
max_iters: 600
decay_lr: True
min_lr: 6e-05
grad_clip: 1.0
seed_offset: 0
device: cpu
dtype: float16
compile: True
model_name: Impressions

Training...
loading meta data from components/data/ColdFusion/meta.pkl
found vocab_size = 12532 (inside components/data/ColdFusion/meta.pkl)
number of parameters: 1.15M


/opt/conda/lib/python3.10/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


compiling the model... (takes a ~minute)


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


step 0: train loss 9.4520, val loss 9.4510
Checkpoint saved at checkpoints/Iter_0.pth
Step: 0, Loss: 9.4547, Learning Rate: 0.000600
Step: 1, Loss: 9.4009, Learning Rate: 0.000600
Step: 2, Loss: 9.3398, Learning Rate: 0.000600
Step: 3, Loss: 9.2697, Learning Rate: 0.000600
Step: 4, Loss: 9.2323, Learning Rate: 0.000600
Step: 5, Loss: 9.1892, Learning Rate: 0.000600
Step: 6, Loss: 9.1553, Learning Rate: 0.000600
Step: 7, Loss: 9.1066, Learning Rate: 0.000600
Step: 8, Loss: 9.0620, Learning Rate: 0.000600
Step: 9, Loss: 9.0119, Learning Rate: 0.000600
Step: 10, Loss: 8.9736, Learning Rate: 0.000600
Step: 11, Loss: 8.9165, Learning Rate: 0.000600
Step: 12, Loss: 8.8580, Learning Rate: 0.000600
Step: 13, Loss: 8.8424, Learning Rate: 0.000600
Step: 14, Loss: 8.7853, Learning Rate: 0.000600
Step: 15, Loss: 8.7363, Learning Rate: 0.000600
Step: 16, Loss: 8.6970, Learning Rate: 0.000600
Step: 17, Loss: 8.6886, Learning Rate: 0.000600
Step: 18, Loss: 8.5992, Learning Rate: 0.000600
Step: 19, Lo

In [15]:
from components.data import tokenizer
tokenizer_instance = tokenizer.TextTokenizer(tokenizer_type=trainer.meta['tokenizer_type'], stoi=trainer.stoi, itos=trainer.itos)

initial_text = "chat GPT moment"
encoded_input = tokenizer_instance.encode(initial_text)
context = torch.tensor(encoded_input, dtype=torch.long, device=config.device)
context = context.unsqueeze(0)
generated_encoded_text = model.generate(context, max_new_tokens=50)[0].tolist()
print(tokenizer_instance.decode(generated_encoded_text))

The tokenizer type is : gpt-4
This is the encoded numbers : [3640, 227, 1272, 1886, 111, 84, 3213, 2065, 805, 799, 392, 12140, 4682, 190, 7963, 7696, 116, 7104, 109, 7038, 1176, 165, 1074, 2829, 1027, 284, 5382, 4556, 352, 11887, 84, 481, 84, 10837, 507, 293, 328, 165, 422, 856, 167, 6666, 411, 1160, 3576, 109, 2746, 5234, 192, 549, 1749, 70, 12238, 103]
chat GPT moment of the Services seems well model so foundational ourselves that barriers economics and cycles toMusic might be version billion level was fraud imagine willmitters theuth thehythm when we can be one think on iPad.S chang visible to190 languages it like vo a degraded in


In [10]:
encoded_input

[27, 27, 293, 63]

In [11]:
# import argparse
# from components.config.Config import ConfigGPT, override_config_with_args

# def parse_args():
#     parser = argparse.ArgumentParser(description="Training Configuration Overrides")

#     # General settings
#     parser.add_argument('--random_seed', type=int, help='Random seed for reproducibility')
#     parser.add_argument('--device', type=str, choices=['cuda', 'cpu'], help='Device to run the training on')

#     # Data settings
#     parser.add_argument('--dataset_path', type=str, help='Path to the dataset')
#     parser.add_argument('--batch_size', type=int, help='Batch size for training')
#     parser.add_argument('--num_workers', type=int, help='Number of data loading workers')

#     # Model settings
#     parser.add_argument('--model_name', type=str, help='Model architecture to use')
#     parser.add_argument('--pretrained', type=bool, help='Use pretrained model weights')
#     parser.add_argument('--num_classes', type=int, help='Number of output classes')

#     # Training settings
#     parser.add_argument('--learning_rate', type=float, help='Learning rate for the optimizer')
#     parser.add_argument('--epochs', type=int, help='Number of training epochs')
#     parser.add_argument('--weight_decay', type=float, help='Weight decay (L2 regularization)')

#     return parser.parse_args()

# def main():
#     # Load default configuration
#     config = ConfigGPT()
#     print(config)

#     # Parse command-line arguments
#     args = parse_args()

#     # Override configuration with command-line arguments
#     override_config_with_args(config, args)

#     # Display the final configuration
#     print(config)

#     # Here, you would normally continue with the rest of your training script
#     # For example:
#     # train_model(config)

# if __name__ == "__main__":
#     main()